# Problem - Part One
Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:

    pbga (66)
    xhth (57)
    ebii (61)
    havc (66)
    ktlj (57)
    fwft (72) -> ktlj, cntj, xhth
    qoyq (66)
    padx (45) -> pbga, havc, qoyq
    tknk (41) -> ugml, padx, fwft
    jptl (61)
    ugml (68) -> gyxo, ebii, jptl
    gyxo (61)
    cntj (57)

...then you would be able to recreate the structure of the towers that looks like this:

                    gyxo
                  /     
             ugml - ebii
           /      \     
          |         jptl
          |             
          |         pbga
         /        /     
    tknk --- padx - havc
         \        \     
          |         qoyq
          |             
          |         ktlj
           \      /     
             fwft - cntj
                  \     
                    xhth

In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

## Solution
First we need to be able to store our data somehow. Since this has more parsing information than previous problems, we'll define this first up front. Each line of data in the input holds the data `NAME (WEIGHT)[-> CHILDREN]` - so we'll just parse those into a tuple with that data.

In [23]:
from collections import namedtuple


Program = namedtuple("Program", ["name", "weight", "children"])


def parse(line):
    prog_data, child_data = line.split(" -> ") if " -> " in line else [line, None]
    name, weight_data     = prog_data.split()
    weight                = int(weight_data.replace("(","").replace(")",""))

    if child_data:
        children = [n.strip() for n in child_data.split(",")]
    else:
        children = []
    
    return Program(name, weight, children)

In [24]:
[parse(line) for line in open("./input.txt")]

[Program(name='jovejmr', weight=40, children=[]),
 Program(name='fesmk', weight=24, children=[]),
 Program(name='gwhfv', weight=74, children=[]),
 Program(name='vxfoyx', weight=101, children=['aqytxb', 'ltnnn']),
 Program(name='pvtnv', weight=77, children=[]),
 Program(name='cpmuhnf', weight=39, children=[]),
 Program(name='ocezven', weight=77, children=[]),
 Program(name='xjqta', weight=42, children=[]),
 Program(name='hkckef', weight=21, children=[]),
 Program(name='nwuhqn', weight=63, children=[]),
 Program(name='wbbfc', weight=49, children=[]),
 Program(name='oictp', weight=50, children=[]),
 Program(name='zmizid', weight=85, children=[]),
 Program(name='uukembw', weight=1054, children=['yowehs', 'ytoju', 'jwyhe', 'bxqlx']),
 Program(name='iqdna', weight=212, children=['gmhwcj', 'vllsfc', 'ebptuar', 'lmcqa']),
 Program(name='vakapy', weight=132, children=['eiisyk', 'nghznvw', 'dqslnjk']),
 Program(name='qouck', weight=77, children=[]),
 Program(name='pqjeof', weight=12, children=[]

Finding the root node should be fairly straightforward - we just need to know which programs have no parents. In order to do so, we can just go through all of the programs and store the inverted child->parent relationship into a dictionary called `parents`.

We'll also store the program data by name for convenience later on.

In [97]:
from collections import defaultdict

programs = {}
parents = {}
for line in open("./input.txt"):
    program = parse(line)
    
    programs[program.name] = program
    parents.update({child: program for child in program.children})

root = list(set(programs) - set(parents))[0]
root

'ahnofa'

# Problem - Part Two
The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.

In the example above, this means that for ugml's disc to be balanced, gyxo, ebii, and jptl must all have the same weight, and they do: 61.

However, for tknk to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same:

    ugml + (gyxo + ebii + jptl) = 68 + (61 + 61 + 61) = 251
    padx + (pbga + havc + qoyq) = 45 + (66 + 66 + 66) = 243
    fwft + (ktlj + cntj + xhth) = 72 + (57 + 57 + 57) = 243

As you can see, tknk's disc is unbalanced: ugml's stack is heavier than the other two. Even though the nodes above ugml are balanced, ugml itself is too heavy: it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced. If this change were made, its weight would be 60.

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

## Solution

In [57]:
def weight(node):
    return node.weight + sum([weight(programs[child]) for child in node.children])

[weight(programs[child]) for child in programs[root].children]

[69875, 69875, 69875, 69875, 69875, 69881]

In [98]:
def find_offbalanced_node(node):    
    if node.children:
        weights = [weight(programs[child]) for child in node.children]
        counts = {x: weights.count(x) for x in weights}
        
        if len(counts) == 1:
            return node
        else:
            off_weight = min(counts, key=lambda key: counts[key])
            off_node = node.children[weights.index(off_weight)]
            return find_offbalanced_node(programs[off_node])
    else:
        return node

In [99]:
offbalanced_node = find_offbalanced_node(programs[root])
parent = parents[offbalanced_node.name]
print("Offbalanced Node: {0}".format(offbalanced_node.name))
print("Neighbors   Weights: {0}".format(
    [(child, weight(programs[child])) for child in parent.children]
))
print("Children of Offbalanced Node: {0}".format(
    [(child, weight(programs[child])) for child in offbalanced_node.children]
))

Offbalanced Node: ltleg
Neighbors   Weights: [('jnyexah', 1786), ('ltleg', 1792), ('fdnmqri', 1786), ('iysgr', 1786), ('dffdie', 1786), ('vvgyhb', 1786)]
Children of Offbalanced Node: [('vnlmjy', 164), ('bscjk', 164), ('uycjw', 164), ('kzuimi', 164), ('kwlqal', 164), ('lahieha', 164)]


In [96]:
def solve(root_node):
    offbalanced_node = find_offbalanced_node(root_node)
    parents[offbalanced_node.name]
    neighbors = [programs[child] for child in parent.children if child != offbalanced_node.name]
    weight_diff = weight(neighbors[0]) - weight(offbalanced_node)
    return offbalanced_node.weight + weight_diff

solve(programs[root])

802